In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from matplotlib.animation import FuncAnimation

from simulation.emit_particles import getDistances, getDirectedSpread
from simulation.loader import load_environment
from simulation.particle_spread import particle_spread
from simulation.visibility import getVisibilityMaps
from simulation.visualizations import plot_path

In [ ]:
resolution = 3
input_map_path = Path("../data/chalmers_karresturang_normal.txt")


environment = load_environment(str(input_map_path), resolution)
width, height = environment.shape
plot_path(environment)

In [ ]:
visibility_map_path = input_map_path.with_suffix(".visibility.npy")
if visibility_map_path.exists():
    visibilities = np.load(visibility_map_path, allow_pickle=True)
else:
    visibilities = getVisibilityMaps(environment, resolution, screen=False)
    np.save(visibility_map_path, visibilities)

distances = getDistances(visibilities, environment, resolution)
cones = getDirectedSpread(visibilities, environment, resolution)

In [ ]:
density = np.zeros((width, height))

def infection_step(particle_density, infected):
    return particle_spread(resolution=resolution,
                                       environment=environment,
                                       visibilityMatrix=visibilities,
                                       particleMatrix=particle_density,
                                       distanceMatrix=distances,
                                       emissionRate=3).emit(infected, None, cones)

plot_path(environment)

density = infection_step(density, [(50, 40)])
plt.imshow(density.T)
plt.show()

plt.plot(density[:, 40])
plt.show()

x = 1.0 - distances / 2
x[x <= 0] = 0

plt.imshow(x, cmap="Reds")
plt.show()

plt.plot(x[:, 12])
plt.show()

plt.imshow(visibilities[100, 4].T)
plt.show()

# Modelling of risk density

There are many ways to model the spreading of a disease.

-- picture of research papers

For example, individual particles can be simulated. (They interact with the airflow and move around. One can even
differentiate between different sizes which affect how long they stay in the air.)

In order to have a simple model which still captures the main attributes, we assume that distance is the main risk factor.

People may breath, cough or touch things in irregular intervals and we use the risk as an simplification of all that.

...

The most dangerous place is directly next to an infected agent. The more distance, the less risk is associated.

We use 2 meters as the maximum distance that a infected agent can affect others.

...

Spread is not going to happen through a wall. Therefore, we check for walls.

During sitting, the agent is most likely speaking forwards or sideways. Here we use cones.

...



Next up, Mathias is going to talk about the results.

In [ ]:
def plot_agent_walk():
    fig, ax = plt.subplots()
    walk = [(105, y) for y in range(80, 5-1, -1)] + [(x, 5) for x in range(105, 90, -1)]
    m = ax.imshow(visibilities[walk[0]][1:,1:], cmap="Blues_r")

    ax.axis("off")

    ax.add_patch(plt.Circle((12, 12), resolution / 4, fc='k', ec='g', fill=True))
    # ax.plot([12], [12], "kx")

    fig.tight_layout()

    def update(frame):
        m.set_data(visibilities[walk[frame]].T[1:,1:])
        return m,

    animation = FuncAnimation(fig, update, frames=len(walk))
    animation.save("walk.mp4", fps=20, dpi=300)

plot_agent_walk()